Python libraries used in this notebook.

In [1]:
import pandas as pd
import requests


Generate map of census tracts for the 100th state congress district of Florida

In [2]:
resp = requests.get('https://api.census.gov/data/2020/dec/pl?get=NAME&for=tract%20(or%20part):*&in=state:12%20state%20legislative%20district%20(lower%20chamber):100%20county%20(or%20part):086')
txt = resp.json()
map = pd.DataFrame(txt)

In [3]:
new_header = map.iloc[0] 
map = map[1:] 
map.columns = new_header 
map.reset_index(inplace=True, drop=True)

In [4]:
map['CensusTract'] = map['NAME']

In [5]:
map['CensusTract'] = map['CensusTract'].str.replace("Census Tract ","") 
map['CensusTract'] = map['CensusTract'].str.split(',').str[0]
map['CensusTract'] = map['CensusTract'].str.replace("\(part\)","") 

/tmp/ipykernel_62685/2880120312.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  map['CensusTract'] = map['CensusTract'].str.replace("\(part\)","")


Cleanup census age and gender demographics.

In [6]:
df = pd.read_excel('ACSST5Y2020.S0101-2022-07-11T025947.xlsx')
df['CensusTract'] = df['CensusTract'].str.replace("Census Tract","") 
df['CensusTract'] = df['CensusTract'].str.replace(", Miami-Dade County, Florida","")
df['CensusTract'] = df['CensusTract'].str.strip()
df = df.set_index('CensusTract')

In [7]:
df.index = pd.Series(df.index).fillna(method='ffill')

In [8]:
values = ['Total', 'Percent', 'Percent Female', 'Percent Male']

for i in values:
    df = df[df.AgeGroup != i]

In [9]:
df.rename({'AgeGroup': 'Gender'}, axis=1, inplace=True)

In [10]:
df = df.melt(id_vars=['Gender'], var_name='AgeGroup', value_name='Count', ignore_index=False)

filter age and gender demo by florida 100th census tracts

In [11]:
df = df[df.index.isin(map['CensusTract'])]

In [12]:
df.to_excel("output_age_sex_miami_dade_100th.xlsx", sheet_name='Data')